### 判別器最後一層去掉sigmoid
### 生成器和判別器的loss不再取log
### weight clipping: 對參數w做一個限制，如果發現w大於某個常數c，就令w = c; 如果w<-c , w = -c，等於說將參數限制在某一個範圍內。(目的是為了限制discriminator不要變化得太劇烈，盡量平滑）
### 不要用基於動量的優化算法（momentum、Adam等），是一個tricks

In [8]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../MNIST_data/", one_hot=True)

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
import numpy as np
np.set_printoptions(suppress=True) 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [3]:
for folder in ['out/', 'model/']:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [4]:
mb_size = 32
Z_dim = 10
X_dim = mnist.train.images.shape[1] # 784
n_disc = 5

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1./tf.sqrt(in_dim/2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

X = tf.placeholder(tf.float32, shape=[None, X_dim])

D_W1 = tf.Variable(xavier_init([X_dim, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

Z = tf.placeholder(tf.float32, shape=[None, Z_dim])

G_W1 = tf.Variable(xavier_init([Z_dim, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

def discriminator(X):
    D_h1 = tf.nn.relu(tf.matmul(X, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    return D_logit # 不使用 sigmoid

def sample_Z(m,n):
    return np.random.uniform(-1., 1., size=[m,n])

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    return fig

G_sample = generator(Z)
D_real = discriminator(X)
D_fake = discriminator(G_sample)

D_loss = -(tf.reduce_mean(D_real) - tf.reduce_mean(D_fake)) # D_real 與 D_fake 越遠越好
G_loss = -tf.reduce_mean(D_fake)

D_solver = tf.train.RMSPropOptimizer(learning_rate=0.0001).minimize(D_loss, var_list=theta_D) # 使用 RMSProp
G_solver = tf.train.RMSPropOptimizer(learning_rate=0.0001).minimize(G_loss, var_list=theta_G)

clip_D = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in theta_D] # clip

W0303 15:31:33.818741 4507657664 deprecation.py:506] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/python/training/rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0303 15:31:33.824501 4507657664 deprecation.py:506] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/python/training/slot_creator.py:187: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()
# saver.restore(sess, "model/save_net.ckpt")

In [6]:
i=0
for it in range(1000000):   
    for _ in range(n_disc):
        X_mb, _ =  mnist.train.next_batch(mb_size)
        _, D_l, _ = sess.run([D_solver, D_loss, clip_D], feed_dict={X:X_mb, Z:sample_Z(mb_size, Z_dim)})
        
    _, G_l = sess.run([G_solver, G_loss], feed_dict={Z:sample_Z(mb_size, Z_dim)})
    
    if it%1000==0:
        print('iter:{} [D loss:{:.4}] [G loss:{:.4}]'.format(it, D_l, G_l))
        
    if it%1000==0:
        samples = sess.run(G_sample, feed_dict={Z:sample_Z(16, Z_dim)})
        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        plt.close(fig)
        i+=1
        
    if it%1000==0:
        save_path = saver.save(sess, "model/save_net.ckpt")

iter:0 [D loss:-0.004315] [G loss:0.005486]
iter:1000 [D loss:-0.09426] [G loss:0.1435]
iter:2000 [D loss:-0.02234] [G loss:-0.07509]
iter:3000 [D loss:-0.006946] [G loss:-0.03022]
iter:4000 [D loss:-0.06785] [G loss:-0.04262]
iter:5000 [D loss:-0.07439] [G loss:-0.01546]
iter:6000 [D loss:-0.03988] [G loss:-0.006372]
iter:7000 [D loss:-0.0348] [G loss:-0.01137]
iter:8000 [D loss:-0.03561] [G loss:-0.007076]
iter:9000 [D loss:-0.03613] [G loss:-0.006813]
iter:10000 [D loss:-0.03067] [G loss:-0.006473]
iter:11000 [D loss:-0.0305] [G loss:-0.01087]
iter:12000 [D loss:-0.02704] [G loss:-0.01148]
iter:13000 [D loss:-0.02646] [G loss:-0.01217]
iter:14000 [D loss:-0.02166] [G loss:-0.006742]
iter:15000 [D loss:-0.02606] [G loss:-0.005819]
iter:16000 [D loss:-0.02863] [G loss:-0.005403]
iter:17000 [D loss:-0.02597] [G loss:-0.008341]


KeyboardInterrupt: 